In [75]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
#para request
import requests
from getpass import getpass
import parametros
import time

### Definiendo fechas para rangos de consultas

In [76]:
now = datetime.now()
new_date = now - timedelta(days=6)
#fecha_url = str(new_date.strftime(format))
ahora = str(now.strftime("%Y/%m/%d %H:%M"))
fecha_6 = str(new_date.strftime("%Y/%m/%d %H:%M"))
fecha_hoy = str(now.strftime("%Y.%m.%d"))

## Leyendo datos del indice ohmyfi-valoraciones

In [77]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

## tomando fecha mas reciente del indice

In [78]:
total_docs = 1
try:
    response = es.search(
        index= parametros.mintic_control,
        body={
               "_source": ["usuarios.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia_ohmyfi_valoraciones"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['usuarios.fechaControl']
except:
    pass
if response["hits"]["hits"] == []:
    try:
        response = es.search(
            index=parametros.ohmyfi_val_index,
            body={"aggs" : {
                       "max_date": {"max": {"field": "fechahora", "format": "yyyy-MM-dd HH:mm:ss"}}
                    }
                 },
            size=total_docs
        )
        elastic_docs = response["aggregations"]
        fecha_ejecucion = response["aggregations"]["max_date"]['value_as_string']
        if response["hits"]["hits"] == []:
            fecha_ejecucion = "2021-04-01 00:00:00"
    except:
        print("Error calculando rangos") 
        fecha_ejecucion = "2021-04-01 00:00:00"
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-10-20 12:08:41


## Leyendo la APi valoraciones

In [79]:
#url2 = parametros.url_ohmyfi +'?/consumos/=&apiKey=' + parametros.ohmyfi_api_key 
url2 = parametros.url_ohmyfi + '?/valoraciones' + '/=&apiKey=' + parametros.ohmyfi_api_key 

In [80]:
url2

'https://www.ohmyfi.com/ApiOMF?/valoraciones/=&apiKey=okMOpLAkiYpafQKXhXirwUys'

In [81]:
datos_api = pd.DataFrame(columns=["mac_usuario","fecha_resp","hora_resp","id_pregunta","pregunta","respuesta"
                                  ,"lugar_serespondio","lugar_nombre","lugar_cod","nombre_completo","apellidos","correo"
                                  ,"documento","tipo_documento","celular","telefono","direccion","ciudad","pais"
                                  ,"genero","fecha_nacimiento","nivel_educativo","rango_edad","creado"
                                  ,"tipo_dispositivo","sistema_operativo","mac_ap"])

In [82]:
h_inicio = '2021/10/20 11:00' #fecha_ejecucion[0:-3].replace("-","/")
h_fin = '2021/10/20 11:30' #str(( datetime.strptime(h_inicio, '%Y/%m/%d %H:%M') + timedelta(minutes=59)).strftime("%Y/%m/%d %H:%M"))

In [83]:
if h_inicio[8:10] != h_fin[8:10]:
    h_inicio = h_inicio[0:11] + '23:00'
    h_fin = h_inicio[0:11] + '23:59'
url = url2 + '&inicio=' + str(h_inicio) + '&final=' + str(h_fin) 
print(url)
r = requests.get(url)
if r.status_code == 200:
    res = json.loads(r.text)
    #print(url)
    datos_api = datos_api.append(res, ignore_index=True)
    datos_api.drop_duplicates(inplace=True)
else:
    if (r.status_code != 500) & (r.status_code != 204):
        print("Se rompe bucle por error de request horas:",r.status_code)
    else:
        print(h_inicio,"- No hay datos para mostrar.",r.status_code)

https://www.ohmyfi.com/ApiOMF?/valoraciones/=&apiKey=okMOpLAkiYpafQKXhXirwUys&inicio=2021/10/20 11:00&final=2021/10/20 11:30


In [84]:
url

'https://www.ohmyfi.com/ApiOMF?/valoraciones/=&apiKey=okMOpLAkiYpafQKXhXirwUys&inicio=2021/10/20 11:00&final=2021/10/20 11:30'

Se actualiza la fecha de ejecución

In [69]:
if r.status_code in (200,204):
    ## Se actualiza la fecha control
    fecha_ejecucion = (datetime.strptime(h_inicio+':00', '%Y/%m/%d %H:%M:%S')+timedelta(hours=1)).strftime("%Y-%m-%d %H:%M:%S")
    if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S')):
        fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))
    response = es.index(
            index = 'control_valoraciones',
            id = 'jerarquia_ohmyfi_valoraciones',
            body = { 'jerarquia_ohmyfi_valoraciones': 'valoraciones_ohmyfi','usuarios.fechaControl' : fecha_ejecucion}
    )
    print("actualizada fecha control de ejecucion:",fecha_ejecucion)

actualizada fecha control de ejecucion: 2021-10-20 12:30:07


In [70]:
datos_api

,mac_usuario,fecha_resp,hora_resp,id_pregunta,pregunta,respuesta,lugar_serespondio,lugar_nombre,lugar_cod,nombre_completo,...,ciudad,pais,genero,fecha_nacimiento,nivel_educativo,rango_edad,creado,tipo_dispositivo,sistema_operativo,mac_ap
0,64-A2-00-8B-3B-82,2021-10-20,12:30:22.00829,3472,Califica cómo fue tu experiencia en el WiFi en...,Excelente,9169,CERROPERALTA_RIOHACHA,37559-ZIFV566,Edier,...,None,None,m,None,Bachillerato,De 18 a 25 años,2021-09-09 15:57:28.372497-05,Smartphone,Android,NaN
1,40-45-DA-CF-97-D9,2021-10-20,12:30:21.947779,3471,Te gustaría calificar tu última conexión en,Si,6939,EL SALOBRE_RIO DE ORO,30797-ZGYO754,Wendy,...,None,None,f,None,Bachillerato,De 14 a 17 años,2021-06-16 09:39:36.969345-05,Smartphone,Android,NaN
2,20-32-6C-F8-64-A4,2021-10-20,12:30:21.790877,3471,Te gustaría calificar tu última conexión en,Si,7452,CALLE 7 409_CANDELARIA,22486-ZZZY553,Yerson David,...,None,None,m,None,Bachillerato,De 14 a 17 años,2021-06-26 21:58:27.675492-05,Smartphone,Android,NaN
3,02-8F-6A-28-42-02,2021-10-20,12:30:21.772914,3471,Te gustaría calificar tu última conexión en,Si,9582,CORREGIMIENTO VOLADOR_TIERRALTA,32543-KPWM833,Juan david,...,None,None,m,None,Sin estudios,De 14 a 17 años,2021-10-14 20:51:27.274694-05,Smartphone,AndroidOS,NaN
4,D0-1C-3C-6F-88-05,2021-10-20,12:30:21.647408,3472,Califica cómo fue tu experiencia en el WiFi en...,Buena,5879,VDA EL POLVERO_LORICA,31651-ZGYO321,Luis Gabriel,...,None,None,m,None,Posgrado,De 18 a 25 años,2021-07-07 10:51:03.557207-05,Smartphone,Android,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653,5E-35-B3-D0-BC-2D,2021-10-20,12:08:00.781475,3471,Te gustaría calificar tu última conexión en,Si,7475,PALMITO PICAO_ MONTERIA,31004-ZZZY580,Leidy sofia,...,None,None,f,None,Bachillerato,De 18 a 25 años,2021-08-15 09:58:17.016817-05,Smartphone,Android,NaN
3654,C0-D2-DD-8F-76-29,2021-10-20,12:08:00.716645,3471,Te gustaría calificar tu última conexión en,No,9221,CASERIO EL COROSO_SAN PELAYO,32477-ZIFV119,Lisanela,...,None,None,f,None,Bachillerato,De 14 a 17 años,2021-09-12 09:09:42.859078-05,Smartphone,Android,NaN
3655,C0-8C-71-77-81-68,2021-10-20,12:08:00.489744,3471,Te gustaría calificar tu última conexión en,Si,5917,LOS CANOS_SAN CARLOS,32420-ZGYO496,Melissa,...,None,None,f,None,Técnico,De 18 a 25 años,2021-04-25 15:49:25.3278-05,Smartphone,Android,NaN
3656,7C-FD-6B-99-17-15,2021-10-20,12:08:00.189695,3472,Califica cómo fue tu experiencia en el WiFi en...,Buena,8527,VDA EL CEDRO_LA PLATA,36730-VZVF805,Jhon,...,None,None,m,None,Bachillerato,De 14 a 17 años,2021-08-09 13:59:53.008869-05,Smartphone,Android,NaN


In [71]:
# datos_api.to_excel("devices.xlsx")

### Descartando datos de la API que ya están en el indice
* se quito la fecha hoy (solicitado por Napoleon [GM] )


In [72]:
indice = 'valoraciones_ohmyfi_pruebaingesta_20_10_21'

In [73]:
use_these_keys = ["mac_usuario","fecha_resp","hora_resp","id_pregunta","pregunta","respuesta"
                  ,"lugar_serespondio","lugar_nombre","lugar_cod","nombre_completo","apellidos","correo"
                  ,"documento","tipo_documento","celular","telefono","direccion","ciudad","pais"
                  ,"genero","fecha_nacimiento","nivel_educativo","rango_edad","creado"
                  ,"tipo_dispositivo","sistema_operativo","mac_ap",'fechahora','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{str(document['mac_usuario']) + '-' + str(document['fechahora'])}",
                "_source": filterKeys(document),
            }

Se limpia y formatea datos

In [74]:
try:
    datos_api.fillna('', inplace=True)
    datos_api['creado_new'] = datos_api["creado"].str.split(".", n = 1, expand = True)[0] 
    datos_api.drop(columns=['creado'], inplace=True)
    datos_api.rename(columns={'creado_new': 'creado'}, inplace=True)
    datos_api['hora_resp_new'] = datos_api["hora_resp"].str.split(".", n = 1, expand = True)[0] 
    datos_api.drop(columns=['hora_resp'], inplace=True)
    datos_api.rename(columns={'hora_resp_new': 'hora_resp'}, inplace=True)
    datos_api['fechahora'] = datos_api['fecha_resp'] + ' '+ datos_api["hora_resp"]
    #datos_api = datos_api.drop(datos_api[(datos_api['fechahora']<= fecha_max.replace("/","-"))].index)
    datos_api.fillna('', inplace=True)
    datos_api['@timestamp'] = now.isoformat() 
    salida = helpers.bulk(es, doc_generator(datos_api))
    print("Fecha:",now,"- Total documentos insertados:",salida[0])
except:
    print("Nada por insertar")
    pass

Fecha: 2021-10-20 12:30:07.104349 - Total documentos insertados: 3658
